# Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.linear_model import Ridge
from matplotlib.colors import Normalize
import networkx as nx
import itertools
from tqdm import tqdm
from scipy.signal import welch
import matplotlib.cm as cm
import seaborn as sns
from collections import defaultdict
from sklearn.decomposition import PCA
import json

# Baselines

In [2]:
def scale_spectral_radius(W, target_radius=0.95):
    """
    Scales a matrix W so that its largest eigenvalue magnitude = target_radius.
    """
    eigvals = np.linalg.eigvals(W)
    radius = np.max(np.abs(eigvals))
    if radius == 0:
        return W
    return (W / radius) * target_radius

def augment_state_with_squares(x):
    """
    Given state vector x in R^N, return [ x, x^2, 1 ] in R^(2N+1).
    We'll use this for both training and prediction.
    """
    x_sq = x**2
    return np.concatenate([x, x_sq, [1.0]])  # shape: 2N+1

# FSR

In [3]:
def sigmoid(z: np.ndarray, /) -> np.ndarray:
    """Numerically-stable logistic σ : ℝ → (0, 1)."""
    out = np.empty_like(z, dtype=np.float32)
    np.subtract(0.0, z, out)            # out = −z   (no new allocation)
    np.exp(out, out)                    # out = e^(−z)
    out += 1.0                          # 1 + e^(−z)
    np.reciprocal(out, out)             # 1 / (1 + e^(−z))
    return out


def _sample_frequencies(m: int, mode: str = "log") -> np.ndarray:
    """
    Produce m distinct rotation angles θ_i ∈ (0, π).

    * 'log'  : log-uniform in [10⁰·⁰, 10⁰·⁹] Hz then mapped to θ = 2πfΔt
    * 'lin'  : uniform in (0, π)
    * 'gold' : golden-ratio spacing (deterministic)
    """
    if mode == "log":
        # log-spread over ~1 decade, then map to (0, π)
        f = 10 ** np.linspace(0.0, 0.9, m, dtype=np.float32)
        f /= f.max()
        return np.pi * f
    if mode == "lin":
        return np.random.default_rng().uniform(0.01, np.pi - 0.01, size=m)
    if mode == "gold":
        return (np.mod(np.arange(1, m + 1) * 0.61803398875, 1.0) * (np.pi - 0.02) + 0.01)
    raise ValueError("mode must be {'log','lin','gold'}")

def augment_state_with_squares(x):
    """
    Given state vector x in R^N, return [ x, x^2, 1 ] in R^(2N+1).
    We'll use this for both training and prediction.
    """
    x_sq = x**2
    return np.concatenate([x, x_sq, [1.0]])  # shape: 2N+1


# ---------------------------------------------------------------------
# Resonator reservoir class
# ---------------------------------------------------------------------
class FSR3D:
    """
    Frequency-Selective Resonator Echo-State Network.

    • N even ⇒ m = N/2 damped 2-D rotations (planar oscillators)
    • Exact spectral control: eigenvalues r·e^{±iθ_i}
    • Sparse symmetric nearest-neighbour coupling ε·C
    • Optional static gain profile g_k  (sin→σ)  for heterogeneity
    • Optional quadratic + quadrature feature map in the read-out
    """

    # -----------------------------------------------------------------
    def __init__(
        self,
        reservoir_size: int = 400,      # N (must be even)
        frequency_mode: str = "log",    # how to draw θ_i
        r_damp: float = 0.95,           # attenuation per step
        eps_couple: float = 0.05,       # ε   cross-pair mixing strength
        input_scale: float = 0.5,
        leak_rate: float = 1.0,         # α
        ridge_alpha: float = 1e-6,
        use_gain: bool = True,
        gain_beta: float = 2.0,
        gain_sigmoid: bool = True,
        use_quadratic_feat: bool = True,
        seed: int = 42,
    ):
        if reservoir_size % 2:
            raise ValueError("reservoir_size must be even")
        self.N = reservoir_size
        self.m = reservoir_size // 2
        self.r = float(r_damp)
        self.eps = float(eps_couple)
        self.input_scale = input_scale
        self.alpha = leak_rate
        self.ridge_alpha = ridge_alpha
        self.use_gain = use_gain
        self.beta = gain_beta
        self.sig_gain = gain_sigmoid
        self.use_quad = use_quadratic_feat
        self.seed = seed
        self.freq_mode = frequency_mode

        # matrices and state
        self.W_res: np.ndarray | None = None
        self.W_in: np.ndarray | None = None
        self.W_out: np.ndarray | None = None
        self.g: np.ndarray | None = None
        self.x = np.zeros(self.N, dtype=np.float32)

        # one-off construction
        self._build_reservoir()
        self._build_gain()

    # -----------------------------------------------------------------
    # internal builders
    # -----------------------------------------------------------------
    def _build_reservoir(self):
        """
        Build   W_res = G · (R ⊕ ... ⊕ R  +  εC)   without the gain
        (gain applied separately for cheaper runtime multiplication).
        """
        m, r, eps = self.m, self.r, self.eps
        θ = _sample_frequencies(m, mode=self.freq_mode)

        # ----- block-diagonal damped rotations -----------------------
        R_blocks = np.zeros((self.N, self.N), dtype=np.float32)
        for i, theta in enumerate(θ):
            c, s = np.cos(theta) * r, np.sin(theta) * r
            i0 = 2 * i
            R_blocks[i0, i0] = c
            R_blocks[i0, i0 + 1] = -s
            R_blocks[i0 + 1, i0] = s
            R_blocks[i0 + 1, i0 + 1] = c

        # ----- sparse symmetric coupling -----------------------------
        rng = np.random.default_rng(self.seed)
        C = np.zeros_like(R_blocks)
        for i in range(m - 1):                        # nearest-neighbour chain
            b = rng.standard_normal()
            B = np.array([[b, 0.0], [0.0, b]], dtype=np.float32)
            a0, a1 = 2 * i, 2 * (i + 1)
            # upper block
            C[a0 : a0 + 2, a1 : a1 + 2] = B
            # symmetric lower block
            C[a1 : a1 + 2, a0 : a0 + 2] = B.T

        self.W_res = R_blocks + eps * C

    def _build_gain(self):
        """Static per-neuron gain g_k."""
        if not self.use_gain:
            self.g = np.ones(self.N, dtype=np.float32)
            return
        k_idx = np.arange(self.N, dtype=np.float32)
        raw = self.beta * np.sin(2.0 * np.pi * k_idx / self.N)
        self.g = sigmoid(raw) if self.sig_gain else raw.astype(np.float32)

    # -----------------------------------------------------------------
    # reservoir core
    # -----------------------------------------------------------------
    def _update(self, u_t: np.ndarray):
        pre = self.W_res @ self.x + self.W_in @ u_t
        if self.use_gain:
            pre *= self.g
            # pre = self.g*self.W_res @ self.x + self.W_in @ u_t
        
        else:
            pre = self.W_res @ self.x + self.W_in @ u_t

        new_x = np.tanh(pre)
        self.x = (1.0 - self.alpha) * self.x + self.alpha * new_x

    def reset_state(self):
        self.x.fill(0.0)

    # -----------------------------------------------------------------
    # read-out training
    # -----------------------------------------------------------------
    def fit_readout(self, inputs: np.ndarray, targets: np.ndarray, discard: int = 100):
        """
        Teacher-forcing pass to learn W_out via ridge regression.
        * inputs  shape [T, d_in]
        * targets shape [T, d_out]
        """
        T, d_in = inputs.shape
        if T <= discard + 1:
            raise ValueError("Not enough data")

        # random input weights
        rng = np.random.default_rng(self.seed)
        self.W_in = (
            rng.uniform(-1.0, 1.0, size=(self.N, d_in)) * self.input_scale
        ).astype(np.float32)

        # collect echoed states
        self.reset_state()
        states = []
        for t in range(T):
            self._update(inputs[t])
            if t >= discard:
                states.append(self.x.copy())

        X = np.asarray(states, dtype=np.float32)            # [T−d, N]
        Y = targets[discard:]                               # align

        # feature map
        if self.use_quad:
            X_list = []
            for s in X:
                X_list.append(augment_state_with_squares(s))
            feats = np.array(X_list, dtype=np.float32)

        else:
            feats = X

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(feats, Y)
        self.W_out = reg.coef_.astype(np.float32)

    # -----------------------------------------------------------------
    # autoregressive forecast
    # -----------------------------------------------------------------
    def predict_autoregressive(self, init_u: np.ndarray, n_steps: int) -> np.ndarray:
        if self.W_out is None:
            raise RuntimeError("Call fit_readout() before prediction")

        d_in = init_u.shape[0]
        preds = np.empty((n_steps, self.W_out.shape[0]), dtype=np.float32)

        #self.reset_state()
        u_t = init_u.astype(np.float32).copy()

        for t in range(n_steps):
            self._update(u_t)

            if self.use_quad:
                feat_vec = augment_state_with_squares(self.x)
            else:
                feat_vec = self.x

            y_t = (self.W_out @ feat_vec).astype(np.float32)
            preds[t] = y_t
            u_t = y_t[:d_in]

        return preds
    
    def predict_open_loop(self, inputs: np.ndarray):
        preds = []
        for true_input in inputs:
            self._update(true_input)
            if self.use_quad:
                feat_vec = augment_state_with_squares(self.x)
            else:
                feat_vec = self.x
            out = (self.W_out @ feat_vec).astype(np.float32)
            preds.append(out)
        return preds


### NRMSE

In [4]:
def evaluate_nrmse(all_preds, test_target, horizons):
    """
    Evaluate model performance over multiple prediction horizons
    for teacher-forced single-step forecasting or autoregressive rollout.
    """
    horizon_nrmse = {}
    for horizon in horizons:
        preds = all_preds[:horizon]
        targets = test_target[:horizon]
        squared_errors = (preds - targets) ** 2
        variance = np.var(targets, axis=0)
        variance[variance == 0] = 1e-8  # avoid divide-by-zero
        nrmse = np.sqrt(np.sum(squared_errors) / (horizon * np.sum(variance)))
        horizon_nrmse[horizon] = nrmse
    return horizon_nrmse

### VPT

In [5]:
def compute_valid_prediction_time(y_true, y_pred, test_time, lyapunov_time, threshold=0.4):
    y_mean = np.mean(y_true, axis=0)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))

    error = y_true - y_pred
    squared_error = np.sum(error**2, axis=1)
    delta = squared_error / denom

    idx_exceed = np.where(delta > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = test_time[-1]
    else:
        T_VPT = test_time[idx_exceed[0]]

    ratio = T_VPT / lyapunov_time

    return T_VPT, ratio

### ADev

In [6]:
def compute_attractor_deviation(predictions, targets, cube_size=(0.1, 0.1, 0.1)):
    """
    Compute the Attractor Deviation (ADev) metric.

    Parameters:
        predictions (numpy.ndarray): Predicted trajectories of shape (n, 3).
        targets (numpy.ndarray): True trajectories of shape (n, 3).
        cube_size (tuple): Dimensions of the cube (dx, dy, dz).

    Returns:
        float: The ADev metric.
    """
    # Define the cube grid based on the range of the data and cube size
    min_coords = np.min(np.vstack((predictions, targets)), axis=0)
    max_coords = np.max(np.vstack((predictions, targets)), axis=0)

    # Create a grid of cubes
    grid_shape = ((max_coords - min_coords) / cube_size).astype(int) + 1

    # Initialize the cube occupancy arrays
    pred_cubes = np.zeros(grid_shape, dtype=int)
    target_cubes = np.zeros(grid_shape, dtype=int)

    # Map trajectories to cubes
    pred_indices = ((predictions - min_coords) / cube_size).astype(int)
    target_indices = ((targets - min_coords) / cube_size).astype(int)

    # Mark cubes visited by predictions and targets
    for idx in pred_indices:
        pred_cubes[tuple(idx)] = 1
    for idx in target_indices:
        target_cubes[tuple(idx)] = 1

    # Compute the ADev metric
    adev = np.sum(np.abs(pred_cubes - target_cubes))
    return adev

### PSD

In [7]:
def compute_psd(y, dt=0.01):
    z = y[:, 2]  # Extract Z-component
    
    # Compute PSD using Welch’s method
    freqs, psd = welch(z, fs=1/dt, window='hamming', nperseg=len(z))  # Using Hamming window
    
    return freqs, psd

# MIT-BIH Dataset

In [8]:
def create_delay_embedding(signal, embed_dim):
    L = len(signal) - embed_dim + 1
    emb = np.zeros((L, embed_dim))
    for i in range(L):
        emb[i, :] = signal[i:i+embed_dim]
    return emb

In [9]:
!pip install wfdb

In [10]:
import wfdb

# Download and load record and annotations for patient #100
record = wfdb.rdrecord('100', sampfrom=0, sampto=25002, pn_dir='mitdb')  # first 20,000 samples
annotation = wfdb.rdann('100', 'atr', sampfrom=0, sampto=25002, pn_dir='mitdb')

In [11]:
# Get input signal u(t) from the first channel
u = record.p_signal[:, 0] 
u

array([-0.145, -0.145, -0.145, ..., -0.41 , -0.415, -0.425],
      shape=(25002,))

In [12]:
# Normalize input
u_min = np.min(u)
u_max = np.max(u)
u_norm = (u - u_min) / (u_max - u_min)

In [13]:
fs = record.fs  # sampling frequency (should be 360 Hz)
t_vals = np.arange(len(u_norm)) / fs

In [14]:
emb_dim = 3
# inputs = u_norm
inputs = create_delay_embedding(u_norm, emb_dim)

# Create target array (heartbeat locations)
targets = np.zeros(len(u_norm))
targets[annotation.sample] = 1  # mark annotations as 1 (heartbeat)
targets = create_delay_embedding(targets, emb_dim)

In [15]:
data_size = len(inputs)
train_size = 15000
train_input = inputs[:train_size]
train_target = targets[:train_size]
test_input = inputs[train_size+1:]
test_target = targets[train_size+1:]
test_size = len(test_input)
print(f"Total samples: {data_size}, train size: {train_size}, test size: {test_size}") 

Total samples: 25000, train size: 15000, test size: 9999


In [16]:
horizons = [300, 600, 1000]

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in tqdm(seeds):
    fsr = FSR3D(
        reservoir_size=300,
        frequency_mode='gold',
        r_damp=0.1,
        eps_couple=0.1,
        input_scale=0.01,
        leak_rate=0.74,
        ridge_alpha=1e-08,
        use_gain=True,
        gain_beta=2,
        gain_sigmoid=True,
        use_quadratic_feat=True,
        seed=seed
    )
    fsr.fit_readout(train_input, train_target, discard=5000)
    fsr_preds = fsr.predict_open_loop(test_input)
    fsr_nrmse = evaluate_nrmse(fsr_preds, test_target, horizons)
    nrmse_dict['FSR'].append(fsr_nrmse)

100%|██████████| 30/30 [01:08<00:00,  2.28s/it]


In [17]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'FSR':<17}")
print("-" * 140)

for horizon in horizons:
    fsr_vals = [np.mean(fsr_nrmse[horizon]) for fsr_nrmse in nrmse_dict['FSR']]

    print(f"{horizon:<10}", end=" ")
    for vals in [fsr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
FSR              
--------------------------------------------------------------------------------------------------------------------------------------------
300        0.7964435726838145 ± 0.0011043094920344864
600        0.7980929123604602 ± 0.0007591047033116995
1000       0.8025282217612327 ± 0.0007044631521750918


# Sunspot Dataset

In [18]:
import pandas as pd
file_path = 'RealWorld/datasets/SN_m_tot_V2.0.csv'

df = pd.read_csv(file_path, sep=';', header = None)
df

FileNotFoundError: [Errno 2] No such file or directory: 'RealWorld/datasets/SN_m_tot_V2.0.csv'

In [ ]:
data = df.iloc[:, 3].values
dt = 1
dataset_size = len(data)
data = create_delay_embedding(data, 3)
print(f"Dataset size: {dataset_size}")

# Train/Test Split
train_end = 2000
train_input  = data[:train_end]
train_target = data[1:train_end+1]
test_input   = data[train_end:-1]
test_target  = data[train_end+1:]
y_test = test_target
n_test_steps = len(test_target)
time_test = np.arange(n_test_steps) * dt

print(f"Train size: {len(train_input)}\nTest size: {len(test_input)}")

In [ ]:
horizons = [300, 600, 1000]

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:
    fsr = FSR3D(
        reservoir_size=300,
        frequency_mode='gold',
        r_damp=0.01,
        eps_couple=0.1,
        input_scale=0.5,
        leak_rate=0.1,
        ridge_alpha=1e-08,
        use_gain=True,
        gain_beta=60,
        gain_sigmoid=True,
        use_quadratic_feat=True,
        seed=seed
    )
    fsr.fit_readout(train_input, train_target, discard=100)
    fsr_preds = fsr.predict_open_loop(test_input)
    fsr_nrmse = evaluate_nrmse(fsr_preds, test_target, horizons)
    nrmse_dict['FSR'].append(fsr_nrmse)

In [ ]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'FSR':<17}")
print("-" * 140)

for horizon in horizons:
    fsr_vals = [np.mean(fsr_nrmse[horizon]) for fsr_nrmse in nrmse_dict['FSR']]

    print(f"{horizon:<10}", end=" ")
    for vals in [fsr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()

# Sante Fe Dataset

In [ ]:
import pandas as pd

file_path = 'RealWorld/datasets/santa-fe-time-series-competition-data-set-b-1.0.0/b1.txt'

df = pd.read_csv(file_path, header=None, sep=' ')
df

In [ ]:
# Normalize the first column (column 0) of the DataFrame
df[0] = (df[0] - df[0].min()) / (df[0].max() - df[0].min())

In [ ]:
data = df.iloc[:, 0].values
chosen_system = "SantaFe"
dt = 1
T_data = len(data)
data = create_delay_embedding(data, 3)
print(f"Data length: {T_data}.")

# Train/Test Split
train_end = 7000
train_input  = data[:train_end]
train_target = data[1:train_end+1]
test_input   = data[train_end:-1]
test_target  = data[train_end+1:]
y_test = test_target
n_test_steps = len(test_target)
time_test = np.arange(n_test_steps) * dt

print(f"Train size: {len(train_input)}  \nTest size: {len(test_input)}")


In [ ]:
horizons = [300, 600, 1000]

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:
    fsr = FSR3D(
        reservoir_size=300,
        frequency_mode='gold',
        r_damp=0.01,
        eps_couple=0.1,
        input_scale=0.5,
        leak_rate=0.1,
        ridge_alpha=1e-08,
        use_gain=True,
        gain_beta=60,
        gain_sigmoid=True,
        use_quadratic_feat=True,
        seed=seed
    )
    fsr.fit_readout(train_input, train_target, discard=100)
    fsr_preds = fsr.predict_open_loop(test_input)
    fsr_nrmse = evaluate_nrmse(fsr_preds, test_target, horizons)
    nrmse_dict['FSR'].append(fsr_nrmse)

In [ ]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'FSR':<17}")
print("-" * 140)

for horizon in horizons:
    fsr_vals = [np.mean(fsr_nrmse[horizon]) for fsr_nrmse in nrmse_dict['FSR']]

    print(f"{horizon:<10}", end=" ")
    for vals in [fsr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()

In [ ]:
# for seed in seeds:
#     crj = CRJRes3D(
#         reservoir_size=500,
#         edge_weight=0.8,
#         jump=15,
#         spectral_radius=0.99,
#         input_scale=0.2,
#         leaking_rate=0.8,
#         ridge_alpha=1e-6,
#         seed=seed
#     )
#     crj.fit_readout(train_input, train_target, discard=5000)
#     crj_preds = crj.predict(test_input)
#     crj_nrmse = evaluate_nrmse(crj_preds, test_target, all_horizons)
#     nrmse_dict['CRJ'].append(crj_nrmse)

# for seed in seeds:
#     mci_esn = MCIESN3D(
#         reservoir_size=500,
#         cycle_weight=0.8,
#         connect_weight=0.8,
#         combine_factor=0.1,
#         v1=0.03,
#         v2=0.03,
#         spectral_radius=0.99,
#         leaking_rate=0.8,
#         ridge_alpha=1e-6,
#         seed=seed
#     )
#     mci_esn.fit_readout(train_input, train_target, discard=5000)
#     mci_esn_preds = mci_esn.predict(test_input)
#     mci_esn_nrmse = evaluate_nrmse(mci_esn_preds, test_target, horizons)
#     nrmse_dict['MCI-ESN'].append(mci_esn_nrmse)


# for seed in seeds:
#     deepesn = DeepESN3D(
#         num_layers=5,
#         reservoir_size=100,
#         spectral_radius=0.99,
#         input_scale=0.2,
#         leaking_rate=0.8,
#         ridge_alpha=1e-6,
#         seed=seed
#     )
#     deepesn.fit_readout(train_input, train_target, discard=5000)
#     deepesn_preds = deepesn.predict(test_input)
#     deepesn_nrmse = evaluate_nrmse(deepesn_preds, test_target, horizons)
#     nrmse_dict['DeepESN'].append(deepesn_nrmse)


In [ ]:
for horizon in [1000]:
    crj_vals = [np.mean(crj_nrmse[horizon]) for crj_nrmse in nrmse_dict['CRJ']]

In [ ]:
crj_vals

In [ ]:
nrmse_dict['CRJ']

In [ ]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'HFR':<17}")
print("-" * 140)


for horizon in horizons:
    hfr_vals = [np.mean(hfr_nrmse[horizon]) for hfr_nrmse in nrmse_dict['HFR']]

    print(f"{horizon:<10}", end=" ")
    for vals in [hfr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()